In [ ]:
from pathlib import Path
import re

import matplotlib.pyplot as plt
import numpy as np

from topostats.plottingfuncs import Colormap

colormap = Colormap().get_cmap()

In [ ]:
DATA_DIR = Path("/Users/sylvi/topo_data/hariborings/test_data/")
OFF_TARGET_DATA_DIR = Path("/Users/sylvi/topo_data/hariborings/off_target/")
TRAINING_DATA_DIR = Path("/Users/sylvi/topo_data/hariborings/training_data/")
FLATTENED_IMAGES_DIR = Path("/Users/sylvi/topo_data/hariborings/flattened_images/")

In [ ]:
# SAVE THE IMAGES

# for sample_type in ["on_target", "off_target"]:
#     for sample in (DATA_DIR / sample_type).glob("*.npy"):
#         data = np.load(sample)
#         plt.imshow(data, vmin=-8, vmax=8, cmap=colormap)
#         plt.title(f"{sample_type} {sample.stem}")
#         plt.show()
#         plt.imsave(
#             f"{FLATTENED_IMAGES_DIR / sample_type / sample.stem}.png",
#             data,
#             vmin=-8,
#             vmax=8,
#             cmap=colormap,
#         )

In [ ]:
# Print all data files
# DATA_ON_TARGET = Path("/Users/sylvi/topo_data/hariborings/data/off_target")
DATA_DNA_ONLY = Path(
    "/Volumes/shared/pyne_group/Shared/AFM_Data/Cas9_Minicircles/Analysis_all/output_abs1_1nm_all/DNA_only/ON_REL/processed/"
)
# Check dirctory exists
print(DATA_DNA_ONLY.exists())
TRAINING_DATA_CROPPED_DIR = TRAINING_DATA_DIR / "cropped" / "dna_only" / "images"
# Create directory if it doesn't exist
TRAINING_DATA_CROPPED_DIR.mkdir(parents=True, exist_ok=True)
files_on_target = list(DATA_DNA_ONLY.glob("*.npy"))
for file in files_on_target:
    print(file.stem)

file = files_on_target[4]
image = np.load(file)

plt.imshow(image, cmap=colormap, vmin=-8, vmax=8)
plt.show()

In [ ]:
# Crop the image to get molecules of interest
width = 60
x = 270
y = 400
cropped_molecule = image[y : y + width, x : x + width]
plt.imshow(cropped_molecule, vmin=-8, vmax=8, cmap=colormap)
plt.show()

In [ ]:
import cv2
import cmapy


image_index = 23
file = files_on_target[image_index]
image = np.load(file)

bounding_box_size = 60

# Define the bounding box
x, y, w, h = 100, 100, bounding_box_size, bounding_box_size

while True:
    # Get cropped image
    cropped_image = image[y : y + h, x : x + w]
    cropped_image_rgb = cropped_image.copy()

    # Make a copy of the image
    display_image = image.copy()

    # Turn the heightmap into a color image
    display_image_norm = cv2.normalize(display_image, None, 0, 255, cv2.NORM_MINMAX)
    display_image = cv2.applyColorMap(display_image_norm.astype(np.uint8), cmapy.cmap("afmhot"))

    # Draw the bounding box
    cv2.rectangle(display_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Show the image with the bounding box and also show the cropped image. They cannot be hstacked though because they are different sizes
    cv2.imshow("image", display_image)

    # Apply a colormap to the cropped image where the minimum and maximum are set to the minimum and maximum of the original image
    cropped_image_rgb = cv2.normalize(cropped_image_rgb, None, 0, 255, cv2.NORM_MINMAX)
    cropped_image_rgb = cv2.applyColorMap(cropped_image_rgb.astype(np.uint8), cmapy.cmap("afmhot"))
    cv2.imshow("cropped", cropped_image_rgb)

    # Wait for a key press
    key = cv2.waitKey(1)

    # Move the bounding box
    if key == ord("a"):
        x -= 10
    elif key == ord("d"):
        x += 10
    elif key == ord("w"):
        y -= 10
    elif key == ord("s"):
        y += 10

    # Save the region in the bounding box when space is pressed
    elif key == ord(" "):
        # Get the index of the output file
        output_index = len(list(TRAINING_DATA_CROPPED_DIR.glob("*.npy")))

        # Save the cropped image
        np.save(TRAINING_DATA_CROPPED_DIR / f"image_{output_index}.npy", cropped_image)
        # Save as png

        plt.imsave(
            TRAINING_DATA_CROPPED_DIR / f"image_{output_index}.png",
            cropped_image,
            vmin=image.min(),
            vmax=image.max(),
        )

    # Quit the program when 'q' is pressed
    elif key == ord("q"):
        break

# Clean up
cv2.destroyAllWindows()
cv2.destroyWindow("image")

In [ ]:
# Save the molecule
# Set i to be the number of molecules saved so far
output_index = len(list(TRAINING_DATA_CROPPED_DIR.glob("*.npy")))
np.save(TRAINING_DATA_CROPPED_DIR / f"image_{output_index}.npy", cropped_molecule)
plt.imsave(
    TRAINING_DATA_CROPPED_DIR / f"image_{output_index}.png",
    cropped_molecule,
    vmin=image.min(),
    vmax=image.max(),
)

In [ ]:
# Rename masks

path = Path("/Users/sylvi/topo_data/hariborings/training_data/cropped/masks/")
image_index = 0

files = list(path.glob("*.npy"))
files.sort()
for file in files:
    print(file.name)
    new_filename = path / f"mask_{image_index}.npy"
    file.rename(new_filename)
    image_index += 1

In [ ]:
images = sorted(list(Path("/Users/sylvi/topo_data/hariborings/training_data/cropped/masks/").glob("*.npy")))
print(images[0])
image = np.load(images[0])
plt.imshow(image)
print(np.unique(image))

In [ ]:
# Merge multiclass masks into one mask
MULTICLASS_MASK_DIR = Path("/Users/sylvi/topo_data/hariborings/training_data/cropped/multiclass_masks/")

mask_files = sorted(list(MULTICLASS_MASK_DIR.glob("*.npy")))

# Files with the same task id are the same molecule and need to be merged.
image_index = 0
for task in range(252, 302):
    print(f"task: {task}")
    # Both masks have "task-{task}" in the name.
    # Get the ring mask. It is the only file with both "task-{task}" and "ring" in the name.
    ring_file = [file for file in mask_files if f"task-{task}" in file.name and "ring" in file.name][0]
    gem_file = [file for file in mask_files if f"task-{task}" in file.name and "gem" in file.name][0]
    # print(f"ring_file: {ring_file.stem}, gem_file: {gem_file.stem}")
    ring_mask = np.load(ring_file).astype(bool)
    gem_mask = np.load(gem_file).astype(bool)
    # plt.imshow(ring_mask)
    # plt.show()
    # plt.imshow(gem_mask)
    # plt.show()
    combined_mask = np.zeros_like(ring_mask).astype(int)
    combined_mask[ring_mask] = 1
    combined_mask[gem_mask] = 2
    # plt.imshow(combined_mask)
    # plt.show()
    plt.imsave(MULTICLASS_MASK_DIR / f"mask_{image_index}.png", combined_mask)
    np.save(MULTICLASS_MASK_DIR / f"mask_{image_index}.npy", combined_mask)
    image_index += 1